# Comparaison de notre modèle spécialisé par rapport à des modèles sentiment généraliste

In [35]:
import time
import pandas as pd
from sklearn.metrics import accuracy_score

# Votre modèle
import keras
import keras_nlp

# Modèles de comparaison simples
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
BERT_BASE_MODEL_PATH = '../outputs/models/prod/bert_base_en_model.keras'
PRESET_BASE_NAME = 'bert_base_en_uncased'

BERT_SMALL_MODEL_PATH = '../outputs/models/prod/bert_small_en_model.keras'
PRESET_SMALL_NAME = 'bert_small_en_uncased'

In [37]:
print("Chargement des modèles...\n")

bert_base_model = keras.models.load_model(BERT_BASE_MODEL_PATH)
preprocessor_base = keras_nlp.models.BertPreprocessor.from_preset(PRESET_BASE_NAME, sequence_length=128)

bert_small_model = keras.models.load_model(BERT_SMALL_MODEL_PATH)
preprocessor_small = keras_nlp.models.BertPreprocessor.from_preset(PRESET_SMALL_NAME, sequence_length=128)

# VADER
vader = SentimentIntensityAnalyzer()

Chargement des modèles...



In [38]:
def predict_bert_base_model(text):
    processed = preprocessor_base([text])
    prob = bert_base_model.predict(processed, verbose=0).flatten()[0]
    
    # Prédiction
    prediction = 1 if prob > 0.5 else 0
    
    # Confiance (distance au seuil 0.5)
    confidence = prob if prob > 0.5 else (1 - prob)
    
    return prediction, float(confidence)

def predict_bert_small_model(text):
    processed = preprocessor_small([text])
    prob = bert_small_model.predict(processed, verbose=0).flatten()[0]
    
    # Prédiction
    prediction = 1 if prob > 0.5 else 0
    
    # Confiance (distance au seuil 0.5)
    confidence = prob if prob > 0.5 else (1 - prob)
    
    return prediction, float(confidence)

def predict_vader(text):
    scores = vader.polarity_scores(text)
    compound = scores['compound']
    
    # Prédiction
    prediction = 1 if compound >= 0 else 0
    
    # Confiance (valeur absolue du compound score, normalisée)
    # VADER compound est entre -1 et 1
    # On prend la valeur absolue comme confiance
    confidence = abs(compound)
    
    return prediction, float(confidence)

In [39]:
test_texts = [
    "This game is incredible! I can't stop playing!",
    "Worst purchase ever. Full of bugs and crashes.",
    "It's decent, nothing groundbreaking."
]

for text in test_texts:
    print(f"Review: \"{text}\"")
    
    # VADER
    pred_vader, conf_vader = predict_vader(text)
    sentiment_vader = 'POSITIF' if pred_vader else 'NÉGATIF'
    print(f"  VADER:       {sentiment_vader} (confiance: {conf_vader:.4f} / {conf_vader*100:.2f}%)")
    
    pred_bert, conf_bert = predict_bert_small_model(text)
    sentiment_bert = 'POSITIF' if pred_bert else 'NÉGATIF'
    print(f"  Bert Small Model:  {sentiment_bert} (confiance: {conf_bert:.4f} / {conf_bert*100:.2f}%)")
    
    pred_bert, conf_bert = predict_bert_base_model(text)
    sentiment_bert = 'POSITIF' if pred_bert else 'NÉGATIF'
    print(f"  Bert Base Model:  {sentiment_bert} (confiance: {conf_bert:.4f} / {conf_bert*100:.2f}%)")
    
    print()

Review: "This game is incredible! I can't stop playing!"
  VADER:       POSITIF (confiance: 0.2832 / 28.32%)
  Bert Small Model:  POSITIF (confiance: 0.9839 / 98.39%)
  Bert Base Model:  POSITIF (confiance: 0.9940 / 99.40%)

Review: "Worst purchase ever. Full of bugs and crashes."
  VADER:       NÉGATIF (confiance: 0.6249 / 62.49%)
  Bert Small Model:  NÉGATIF (confiance: 0.9819 / 98.19%)
  Bert Base Model:  NÉGATIF (confiance: 0.9990 / 99.90%)

Review: "It's decent, nothing groundbreaking."
  VADER:       POSITIF (confiance: 0.0000 / 0.00%)
  Bert Small Model:  NÉGATIF (confiance: 0.5259 / 52.59%)
  Bert Base Model:  POSITIF (confiance: 0.7325 / 73.25%)



BERT Base : Comprend "decent" = légèrement positif

BERT Small : Confus par "nothing groundbreaking" (négatif)

VADER : Score exactement neutre (compound = 0.0)

Vocabulaire subtil nécessite modèle large

## Comparaison des approches

### Baseline lexicale (VADER)
- Approche : Dictionnaire de mots positifs/négatifs
- Accuracy : 70%
- Limite : Pas de contexte, échec sur négations

### Transfer Learning générique (DistilBERT-SST2)
- Pré-entraîné : Wikipedia
- Fine-tuné : Critiques de films (SST-2)
- Accuracy : ~85% (estimation)
- Limite : Vocabulaire cinéma non gaming

Installation pytorch / transformer sur un autre projet vscode

### Notre projet (BERT fine-tuné Steam)
- Pré-entraîné : Wikipedia (preset KerasNLP)
- Fine-tuné : 230k reviews Steam (notre dataset)
- Accuracy : 91.90%
- **Gain : +6% vs modèles génériques, +21% vs baseline**

## Conclusion
Le fine-tuning spécialisé sur le domaine gaming apporte une 
amélioration significative face aux approches génériques, 
validant l'intérêt de créer des modèles par domaine.